## ★policy_df 읽어오기(태그생성)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

In [ ]:
policy_df = pd.read_csv("/content/drive/MyDrive/1팀/policy_final_01192022.csv")

In [ ]:
policy_df.columns, policy_df.shape

(Index(['Unnamed: 0', 'Unnamed: 0.1', '정책ID', '정책서비스ID', '서비스명', '소관기관',
        '소관기관유형', '서비스목적', '지원대상', '지원내용', '신청절차', '선정기준', '신청기한_시작', '신청기한_끝',
        '상세조회URL', '문의처전화번호', '온라인신청사이트URL', '지원유형', '추출_종업원수_최소', '추출_종업원수_최대',
        '추출_기업규모', '추출_주요업종', '추출_선호특성', '추출_설립기간_최소', '추출_설립기간_최대', '추출_시도1',
        '추출_시군구1', '주요업종_filtered'],
       dtype='object'), (9630, 28))

# 기업정책데이터 query_answer

## 기업데이터 불러오기

In [ ]:
pd.set_option('display.max_colwidth', None)

df_all = pd.read_excel('/content/drive/MyDrive/1팀/기업데이터 형태샘플(AIB전달용).xlsx', sheet_name = None)
df_all.keys()

dict_keys(['형태샘플', '학습샘플', '최종기업샘플', '10차 업종번호', 'dart_기업개황'])

In [ ]:
df_all['학습샘플'].shape, df_all['형태샘플'].shape, df_all['최종기업샘플'].shape, df_all['dart_기업개황'].shape

((463, 28), (46, 28), (11056, 36), (89390, 19))

In [ ]:
df_all['학습샘플'].columns

Index(['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
       '설립일자', '기업형태', '기업규모', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월', '기업공개일자',
       '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호', '소속그룹',
       '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보'],
      dtype='object')

In [ ]:
df_all['형태샘플'].columns

Index(['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
       '기업체명.1', '설립일자', '기업형태', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월',
       '기업공개일자', '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호',
       '소속그룹', '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보'],
      dtype='object')

In [ ]:
# 잘못된 컬럼명 수정
df_all['형태샘플'].columns = ['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
        '설립일자', '기업형태', '기업규모', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월', '기업공개일자',
        '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호', '소속그룹',
        '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보']
df_all['형태샘플'].head(1)

,섹터,기업체명,영문기업명,사업자번호,법인(주민)번호,대표자명,종업원수,설립형태,설립일자,기업형태,기업규모,전화번호,팩스번호,홈페이지,이메일,결산월,기업공개일자,도로명,지번,업종(10차),업종(9차),주요제품(상품),무역업허가번호,소속그룹,주채권기관,당좌거래은행,휴폐업정보,법인등기정보
0,가구내고용활동및달리분류되지않은자가소비생산활동,(주)루더비즈코리아,"MISERVANT CO.,LTD.",129-81-47474,131111-0070323,김경순,5명,신규설립(개업),2002-01-04 00:00:00,일반법인,소기업,031-719-9777,031-712-2900,www.miservant.com,webmaster@miservant.com,01월,-,(13616) 경기 성남시 분당구 정자일로 1 (금곡동),(13616) 경기 성남시 분당구 금곡동 210번지,(T97000)가구 내 고용활동,(T97000)가구내 고용활동,"개인업무대행서비스,기타",-,-,기업은행,-,폐업자\n(폐업일자 : 2006-01-06),청산종결간주 (조회일자 : 2022-01-03)


In [ ]:
df_sample = pd.concat([df_all['학습샘플'],df_all['형태샘플']], ignore_index=True)

In [ ]:
df_final = df_all['최종기업샘플']

In [ ]:
df_dart = df_all['dart_기업개황']

### 학습샘플+형태샘플 : df_sample

In [ ]:
df_sample.head()

,섹터,기업체명,영문기업명,사업자번호,법인(주민)번호,대표자명,종업원수,설립형태,설립일자,기업형태,기업규모,전화번호,팩스번호,홈페이지,이메일,결산월,기업공개일자,도로명,지번,업종(10차),업종(9차),주요제품(상품),무역업허가번호,소속그룹,주채권기관,당좌거래은행,휴폐업정보,법인등기정보
0,공공행정국방및사회보장행정,금융감독원,Financial Supervisory Service,116-82-14464,116171-0008533,이진석,"1,589명",신규설립(개업),1999-01-02 00:00:00,일반법인,판단제외,02-3145-5293,-,-,-,12월,-,"(07321) 서울 영등포구 여의대로 38 (여의도동,금융감독원)",(07321) 서울 영등포구 여의도동 27번지 금융감독원,(K66110)금융시장 관리업,(K66110)금융시장 관리업,금융기관에 대한 검사.감독 업무 등,-,-,기업은행,우리은행,부가가치세 일반과세자\n(조회일자 : 2022-01-05),정상 (조회일자 : 2021-12-14)
1,공공행정국방및사회보장행정,(주)명현유통,"MYUNG-HYUN DISTRIBUTION co.,ltd.",122-81-66164,124611-0250370,정재운,17명,신규설립(개업),2001-04-26 00:00:00,외감,중기업,070-4940-3676,031-761-6394,myunghyun.store,-,12월,-,(17396) 경기 이천시 호법면 덕평로 236,(17396) 경기 이천시 호법면 매곡리 990-8번지,(G46332)비알코올음료 도매업,(G46332)비알콜음료 도매업,"음료, 잡화, 식자재, 냉동식품 등",-,-,농협은행,농협은행,부가가치세 일반과세자\n(조회일자 : 2022-01-05),정상 (조회일자 : 2022-01-10)
2,공공행정국방및사회보장행정,서울관광마케팅(주),"SEOUL TOURISM MARKETING CO., LTD.",104-86-12627,110111-3836868,이창환,67명,신규설립(개업),2008-02-04 00:00:00,일반법인,중기업,02-3788-0815,02-3788-0899,www.seoulwelcome.com,-,12월,-,"(04551) 서울 중구 삼일대로 340, 9층 (저동1가,나라키움저동빌딩)",(04551) 서울 중구 저동1가 1-2번지 나라키움저동빌딩 9층,(O84212)문화 및 관광 행정,(O84212)문화 및 관광 행정,문화 및 관광 행정,-,-,우리은행,-,폐업자\n(폐업일자 : 2019-07-31),청산종결 (조회일자 : 2019-11-05)
3,공공행정국방및사회보장행정,(주)씨티엘네트웍스,"CTL NETWORKS CO.,LTD.",104-86-01625,110111-3459462,천무진,65명,신규설립(개업),2006-05-17 00:00:00,일반법인,중기업,02-2016-4020,02-3153-2599,www.ctlplus.co.kr,sooya@ectl.net,12월,-,"(06104) 서울 강남구 언주로 651, 8층 (논현동,삼원빌딩)",(06104) 서울 강남구 논현2동 210-1번지 삼원빌딩 8층,(N75210)여행사업,(N75211)일반 및 국외 여행사업,관광서비스 외,-,-,-,-,폐업자\n(폐업일자 : 2015-06-30),해산간주 (조회일자 : 2021-11-04)
4,공공행정국방및사회보장행정,(주)용준,"YONGJUN CO.,LTD.",247-86-01693,200111-0560594,이용창,1명,신규설립(개업),2019-04-17 00:00:00,일반법인,소상공인,062-375-7310,062-375-7333,-,-,12월,-,"(61949) 광주 서구 상무시민로 96, 4층 401호 일부 (치평동,종산타워)",(61949) 광주 서구 치평동 1217-10번지 종산타워 4층 401호 일부,(G47121)슈퍼마켓,(G47121)슈퍼마켓,"슈퍼마켓(와이마트), 건설장비운영, 부동산임대",-,-,국민은행,-,부가가치세 일반과세자\n(조회일자 : 2022-01-04),정상 (조회일자 : 2021-11-12)


In [ ]:
df_sample['설립형태'].value_counts() 

신규설립(개업)    484
기타           10
법인전환          8
분할신설          7
Name: 설립형태, dtype: int64

In [ ]:
df_sample['기업형태'].value_counts() 

일반법인      293
외감        178
개인사업자      23
코스닥시장      10
유가증권시장      4
코넥스         1
Name: 기업형태, dtype: int64

### 최종기업샘플 : df_final

In [ ]:
# df_all['최종기업샘플'].drop('정책select', axis=1, inplace=True) # -->  업종  기업설립일자  종업원수  기업상세주소
df_final.head(5)

,회계감사인명,감사보고서의견내용,기준일자,사업자등록번호,법인구분코드,법인구분코드명,법인영문명,기업체명,법인등록시장구분코드,법인등록시장구분코드명,법인등록번호,종업원평균근속기간내용,기업기본주소,기업상세주소,종업원수,기업설립일자,기업팩스번호,홈페이지,기업코스닥상장폐지일자,기업코스닥상장일자,기업KONEX상장폐지일자,기업KONEX상장일자,기업주요사업명,기업주거래은행명,기업구우편번호,기업공시회사명,기업1인평균급여금액,기업대표자성명,기업결산월,기업전화번호,기업거래소상장폐지일자,기업거래소상장일자,금융감독원법인변경일시,금융감독원법인고유번호,업종,중소기업여부
0,NaN,NaN,20200509,2.118648e+09,NaN,NaN,"DongBang Bldg, 5th Fliir, Suite 506#517, Majang-dong, Seongdong-gu Seoul 133-050",네르프,E,기타,1101111612541,NaN,서울특별시 성동구 마장동 517 동방빌딩 506호,서울특별시 성동구 마장동 517 동방빌딩 506호,0,19981125.0,02.2299.2159,NaN,NaT,NaT,NaN,NaN,NaN,NaN,133050.0,네르프,0,이종경,3.0,02.2299.2150,NaT,NaT,2017-06-30,806149,1,NaN
1,NaN,NaN,20200509,1.208770e+09,NaN,NaN,haemilcommerce,농업회사법인(주)해밀생명과학,E,기타,1101114613760,NaN,서울특별시 강남구 테헤란로84길 14 (대치동)지어러빌딩 7층,서울특별시 강남구 테헤란로84길 14 (대치동)지어러빌딩 7층,0,20110530.0,02-564-7805,NaN,NaT,NaT,NaN,NaN,NaN,NaN,6178.0,농업회사법인해밀생명과학,0,하상욱,12.0,02-564-7801,NaT,NaT,2017-06-30,926629,1,NaN
2,NaN,NaN,20200509,3.028100e+09,NaN,NaN,"SEONG HA CORP. CO.,LTD",주식회사 성하,E,기타,1541110000167,NaN,충청북도 보은군 마로읍 원정리 397,충청북도 보은군 마로읍 원정리 397,0,19560427.0,02-563-1987,없음,NaT,NaT,NaN,NaN,NaN,NaN,135080.0,성하,0,김강원,12.0,02-565-8546,NaT,NaT,2017-06-30,225991,5,NaN
3,NaN,NaN,20200509,2.208747e+09,NaN,NaN,UI holdings,(주)유아이홀딩스,E,기타,1101113661497,NaN,서울특별시 강남구 역삼1동 676 삼부빌딩 9층,서울특별시 강남구 역삼1동 676 삼부빌딩 9층,0,20070420.0,02-527-0706,NaN,NaT,NaT,NaN,NaN,NaN,NaN,135979.0,유아이홀딩스,0,최규선,12.0,02-527-0708,NaT,NaT,2017-06-30,821388,5,NaN
4,NaN,NaN,20200509,2.208756e+09,NaN,NaN,Seven Energy Steel,세븐에너지스틸,E,기타,1101113762287,NaN,서울특별시 강남구 역삼동 690번지,서울특별시 강남구 역삼동 690번지,0,20071002.0,02-538-7972,7egroup.com,NaT,NaT,NaN,NaN,NaN,NaN,135080.0,세븐에너지스틸,0,우태희,12.0,02-566-7077,NaT,NaT,2017-06-30,775061,5,NaN


In [ ]:
df_final['법인등록번호'].isnull().sum()

0

### dart_기업개황 : df_dart

In [ ]:
df_dart.head(5)

,결산월(MM),주소,사업자등록번호,대표자명,"법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)",corp_code,정식명칭,영문명칭,설립일(YYYYMMDD),팩스번호,홈페이지,업종번호,IR홈페이지,법인등록번호,message,전화번호,status,주식 종목코드,종목명 또는 약식명칭
0,12.0,충청남도 천안시 청당동 419-12,3128134722,김상규,E,434003,(주)다코,Daco corporation,19970611.0,041-563-6808,없음,25931,NaN,1615110021778,정상,041-565-1800,0,NaN,다코
1,12.0,강원도 원주시 원동 303,2248105027,이광선,E,434456,일산약품(주),"ILSAN PHARMACEUTICAL CO.,LTD.",19910615.0,033-763-6433,NaN,21101,NaN,1412110004641,정상,033-763-7535,0,NaN,일산약품
2,11.0,서울특별시 중구 태평로2가 120,1048177488,정진형,E,430964,굿앤엘에스주식회사,"Good & LS Co.,Ltd.",20021213.0,NaN,NaN,64999,NaN,1101112672875,정상,02-6263-8185,0,NaN,굿앤엘에스
3,12.0,제주특별자치도 북제주군 한림읍 상명리 587,6168120149,정우석,E,432403,한라판지(주),"HANLA PARKING CO.,LTD",19970601.0,064-796-8003,NaN,17123,NaN,2201110019993,정상,064-796-8001,0,NaN,한라판지
4,10.0,서울특별시 영등포구 여의도동 36-3,107-86-08501,배경오,E,388953,크레디피아제이십오차유동화전문회사,Credipia 25th Asset Securitization Specialty L.L.C,20011121.0,784-9869,NaN,64999,NaN,1101140036118,정상,07-768-7911,0,NaN,크레디피아제이십오차유동화전문회사


In [ ]:
df_dart.columns

Index(['결산월(MM)', '주소', '사업자등록번호', '대표자명',
       '법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)', 'corp_code', '정식명칭', '영문명칭',
       '설립일(YYYYMMDD)', '팩스번호', '홈페이지', '업종번호', 'IR홈페이지', '법인등록번호', 'message',
       '전화번호', 'status', '주식 종목코드', '종목명 또는 약식명칭'],
      dtype='object')

In [ ]:
df_dart['주식 종목코드'].unique()

array([    nan,  36720.,  40130., ...,  53950., 204990., 289220.])

## 기업데이터 컬럼 모아보기

In [ ]:
df_sample.columns

Index(['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
       '설립일자', '기업형태', '기업규모', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월', '기업공개일자',
       '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호', '소속그룹',
       '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보'],
      dtype='object')

In [ ]:
df_final.columns

Index(['회계감사인명', '감사보고서의견내용', '기준일자', '사업자등록번호', '법인구분코드', '법인구분코드명', '법인영문명',
       '기업체명', '법인등록시장구분코드', '법인등록시장구분코드명', '법인등록번호', '종업원평균근속기간내용', '기업기본주소',
       '기업상세주소', '종업원수', '기업설립일자', '기업팩스번호', '홈페이지', '기업코스닥상장폐지일자',
       '기업코스닥상장일자', '기업KONEX상장폐지일자', '기업KONEX상장일자', '기업주요사업명', '기업주거래은행명',
       '기업구우편번호', '기업공시회사명', '기업1인평균급여금액', '기업대표자성명', '기업결산월', '기업전화번호',
       '기업거래소상장폐지일자', '기업거래소상장일자', '금융감독원법인변경일시', '금융감독원법인고유번호', '업종',
       '중소기업여부'],
      dtype='object')

In [ ]:
df_dart.columns

Index(['결산월(MM)', '주소', '사업자등록번호', '대표자명',
       '법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)', 'corp_code', '정식명칭', '영문명칭',
       '설립일(YYYYMMDD)', '팩스번호', '홈페이지', '업종번호', 'IR홈페이지', '법인등록번호', 'message',
       '전화번호', 'status', '주식 종목코드', '종목명 또는 약식명칭'],
      dtype='object')

## 지역 딕셔너리 : region_zip1(시도),  region_zip2(시군구)
- 시도 : region_zip1
- 시군구 : region_zip2

In [ ]:
# 지역명 태그 딕셔너리 만들기

# 1) 시도 태그딕셔너리
policy_tag = pd.read_excel("/content/drive/MyDrive/1팀/정책데이터 태그셋리스트(AIB전달용).xlsx",sheet_name='상위코드', header=1 )
policy_tag = policy_tag.iloc[15:, 4:7]
policy_tag.columns = ['지역(시도)', '번호', '상위코드']
policy_tag.reset_index(drop=True, inplace=True)
region_tag = policy_tag[0:18]
region_tag = region_tag[['지역(시도)','상위코드']]

# 단어와 개체명 정보끼리 딕셔너리 만들기
region_zip = {w: t for w, t in zip(region_tag["지역(시도)"].values.tolist(), region_tag["상위코드"].values.tolist())}
# region_zip_reverse = {t:w for w, t in region_zip.items()}

# 지역이 축약되는 경우도 많으므로 적절하게 처리해서 row 추가 
a = 18
for i in region_tag['지역(시도)']:
    if i == '충청북도':
        region_tag.loc[a] = ['충북', region_zip[i]]
        a+=1
    elif i == '충청남도':
        region_tag.loc[a] = ['충남', region_zip[i]]
        a+=1
    elif i == '전라북도':
        region_tag.loc[a] = ['전북', region_zip[i]]
        a+=1
    elif i == '전라남도':
        region_tag.loc[a] = ['전남', region_zip[i]]
        a+=1
    elif i == '경상북도':
        region_tag.loc[a] = ['경북', region_zip[i]]
        a+=1
    elif i == '경상남도':
        region_tag.loc[a] = ['경남', region_zip[i]]
        a+=1
    else:
        region_tag.loc[a] = [i[0:2], region_zip[i]]
        a+=1

# 단어와 개체명 정보끼리 딕셔너리 만들기
region_zip1 = {w: t for w, t in zip(region_tag["지역(시도)"].values.tolist(), region_tag["상위코드"].values.tolist())}
# region_zip_reverse = {t:w for w, t in region_zip.items()}

# 2) 시군구 딕셔너리 만들고 추가
policy_low_tag= pd.read_excel("/content/drive/MyDrive/1팀/정책데이터 태그셋리스트(AIB전달용).xlsx",sheet_name='하위코드', header=1 )
region_low_tag = policy_low_tag.iloc[1:247,[4,6]].dropna(how='all')
region_low_tag.columns = ['지역(시군구)', '하위코드']
region_low_tag=region_low_tag.reset_index(drop=True)

# 단어와 개체명 정보끼리 쌍으로 묶기. ('전체'는 여러가지라서 따로 처리가 필요)
region_low_tag = region_low_tag[region_low_tag['지역(시군구)'] !='전체']
region_low_zip = {w:t for w, t in zip(region_low_tag["지역(시군구)"].values.tolist(), region_low_tag["하위코드"].values.tolist())}
# region_low_zip_reverse = {t:w for w, t in region_low_zip.items()}

# 지역이 축약되는 경우도 많으므로 적절하게 처리해서 row 추가 
a = 244
for i in region_low_tag['지역(시군구)']:
    if len(i[:-1]) == 1:
        continue
    elif i == '세종시 전체':
        continue
    elif i == '제주시':
        continue
    else:
        region_low_tag.loc[a] = [i[:-1], region_low_zip[i]]
        a+=1

region_zip2 = {w:t for w, t in zip(region_low_tag["지역(시군구)"].values.tolist(), region_low_tag["하위코드"].values.tolist())}
# region_low_zip_reverse = {t:w for w, t in region_low_zip.items()}

# # 두 파이썬 딕셔너리를 병합(시도+시군구)
# region_zip.update(region_low_zip)

In [ ]:
# 시도
region_zip1

{'강원': 'A01-10',
 '강원도': 'A01-10',
 '경기': 'A01-09',
 '경기도': 'A01-09',
 '경남': 'A01-16',
 '경북': 'A01-15',
 '경상남도': 'A01-16',
 '경상북도': 'A01-15',
 '광주': 'A01-05',
 '광주광역시': 'A01-05',
 '대구': 'A01-03',
 '대구광역시': 'A01-03',
 '대전': 'A01-06',
 '대전광역시': 'A01-06',
 '부산': 'A01-02',
 '부산광역시': 'A01-02',
 '서울': 'A01-01',
 '서울특별시': 'A01-01',
 '세종': 'A01-08',
 '세종특별자치시': 'A01-08',
 '울산': 'A01-07',
 '울산광역시': 'A01-07',
 '인천': 'A01-04',
 '인천광역시': 'A01-04',
 '전국': 'A01-00',
 '전남': 'A01-14',
 '전라남도': 'A01-14',
 '전라북도': 'A01-13',
 '전북': 'A01-13',
 '제주': 'A01-17',
 '제주특별자치도': 'A01-17',
 '충남': 'A01-12',
 '충북': 'A01-11',
 '충청남도': 'A01-12',
 '충청북도': 'A01-11'}

In [ ]:
# 시군구
region_zip2

{'가평': 'A01-09-30',
 '가평군': 'A01-09-30',
 '강남': 'A01-01-23',
 '강남구': 'A01-01-23',
 '강동': 'A01-01-25',
 '강동구': 'A01-01-25',
 '강릉': 'A01-10-03',
 '강릉시': 'A01-10-03',
 '강북': 'A01-01-09',
 '강북구': 'A01-01-09',
 '강서': 'A01-02-12',
 '강서구': 'A01-02-12',
 '강진': 'A01-14-13',
 '강진군': 'A01-14-13',
 '강화': 'A01-04-09',
 '강화군': 'A01-04-09',
 '거제': 'A01-16-07',
 '거제시': 'A01-16-07',
 '거창': 'A01-16-17',
 '거창군': 'A01-16-17',
 '경산': 'A01-15-10',
 '경산시': 'A01-15-10',
 '경주': 'A01-15-02',
 '경주시': 'A01-15-02',
 '계룡': 'A01-12-07',
 '계룡시': 'A01-12-07',
 '계양': 'A01-04-07',
 '계양구': 'A01-04-07',
 '고령': 'A01-15-17',
 '고령군': 'A01-15-17',
 '고성': 'A01-16-12',
 '고성군': 'A01-16-12',
 '고양': 'A01-09-03',
 '고양시': 'A01-09-03',
 '고창': 'A01-13-13',
 '고창군': 'A01-13-13',
 '고흥': 'A01-14-09',
 '고흥군': 'A01-14-09',
 '곡성': 'A01-14-07',
 '곡성군': 'A01-14-07',
 '공주': 'A01-12-02',
 '공주시': 'A01-12-02',
 '과천': 'A01-09-29',
 '과천시': 'A01-09-29',
 '관악': 'A01-01-21',
 '관악구': 'A01-01-21',
 '광명': 'A01-09-14',
 '광명시': 'A01-09-14',
 '광산': 'A01-05-0

## df_sample 매칭시스템
- 결측치가 없는 지번으로 매칭

In [ ]:
df_sample.columns

Index(['섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명', '종업원수', '설립형태',
       '설립일자', '기업형태', '기업규모', '전화번호', '팩스번호', '홈페이지', '이메일', '결산월', '기업공개일자',
       '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)', '무역업허가번호', '소속그룹',
       '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보'],
      dtype='object')

### 지역매칭

In [ ]:
policy_df['추출_시도1'].isnull().sum(), policy_df['추출_시군구1'].isnull().sum()

(0, 0)

In [ ]:
# 정책데이터 결측치 제거하고 리스트로 만들기
# policy_df['추출_시군구1'] = policy_df['추출_시군구1'].fillna('')

# policy_df['추출_시도1'] = policy_df['추출_시도'].apply(lambda x : x.split(', '))
# policy_df['추출_시군구1'] = policy_df['추출_시군구'].apply(lambda x:x.split(', '))

In [ ]:
# 어떻게 이루어져 있는지 파악
for i in range(len(df_sample['도로명'])):
    li=df_sample['도로명'][i].split(' ')
    print(li)
    if len(li) < 3:
        print(li)

['(07321)', '서울', '영등포구', '여의대로', '38', '(여의도동,금융감독원)']
['(17396)', '경기', '이천시', '호법면', '덕평로', '236']
['(04551)', '서울', '중구', '삼일대로', '340,', '9층', '(저동1가,나라키움저동빌딩)']
['(06104)', '서울', '강남구', '언주로', '651,', '8층', '(논현동,삼원빌딩)']
['(61949)', '광주', '서구', '상무시민로', '96,', '4층', '401호', '일부', '(치평동,종산타워)']
['(44780)', '울산', '남구', '장생포고래로183번길', '15,', '2층', '(장생포동)']
['(55068)', '전북', '전주시', '완산구', '콩쥐팥쥐로', '1696', '(효자동3가)']
['(26456)', '강원', '원주시', '행구덕현길', '56', '(행구동)']
['(26464)', '강원', '원주시', '세계로', '10', '(반곡동,한국관광공사)']
['(05103)', '서울', '광진구', '자양로', '45', '(자양동)']
['(06349)', '서울', '강남구', '밤고개로1길', '10,', '1123호', '(수서동,수서현대벤처빌)']
['(62395)', '광주', '광산구', '고봉로', '261-4', '(장수동)']
['(14297)', '경기', '광명시', '금오로', '823-21,', '17', '(옥길동)']
['(21693)', '인천', '남동구', '아암대로', '1151', '(고잔동)']
['(12638)', '경기', '여주시', '강변북로', '283,', '7호', '(현암동)']
['-']
['-']
['-']
['-']
['(23011)', '인천', '강화군', '양사면', '인화로', '356', '(인화리)']
['(26496)', '강원', '원주시', '귀래면', '방아실길', '51', '(귀래리)']
['(25948)',

In [ ]:
print(df_sample[df_sample['도로명']=='-'].shape)  # 도로명이 기재되지 않은 회사 17개(지번은 있음)
df_sample[df_sample['도로명']=='-'][['도로명','지번']]

(17, 28)


,도로명,지번
15,-,(38216) 경북 경주시 양남면 환서리 1011번지
16,-,(33504) 충남 보령시 웅천읍 수부리 산6-2번지
38,-,() 강원도 정선군 사북읍 동탄길 38-13
44,-,(38029) 경북 경주시 안강읍 두류리 797번지
51,-,(31962) 충남 서산시 해미면 대곡리 867-5번지
64,-,"(99999) 경기도 성남시 분당구 정자동6 파크뷰제 20층 제상가 2003호, 2004호"
73,-,(36927) 경북 문경시 호계면 선암리 236-1번지
150,-,(12075) 경기 남양주시 진접읍 내곡리 33-16 내 약500평
224,-,(38160) 경북 경주시 인왕동 696-9번지
295,-,(44232) 울산 북구 신명동 104-7번지


In [ ]:
# 지번은 결측치가 없음
for i in range(len(df_sample['지번'])):
    li=df_sample['지번'][i].split(' ')
    print(li)
    if len(li) < 3:
        print(li)

['(07321)', '서울', '영등포구', '여의도동', '27번지', '금융감독원']
['(17396)', '경기', '이천시', '호법면', '매곡리', '990-8번지']
['(04551)', '서울', '중구', '저동1가', '1-2번지', '나라키움저동빌딩', '9층']
['(06104)', '서울', '강남구', '논현2동', '210-1번지', '삼원빌딩', '8층']
['(61949)', '광주', '서구', '치평동', '1217-10번지', '종산타워', '4층', '401호', '일부']
['(44780)', '울산', '남구', '장생포동', '203번지', '2층']
['(55068)', '전북', '전주시', '완산구', '효자동3가', '1052-1번지']
['(26456)', '강원', '원주시', '행구동', '1008-4번지']
['(26464)', '강원', '원주시', '반곡동', '1914-6번지', '한국관광공사']
['(05103)', '서울', '광진구', '자양2동', '679-30번지']
['(06349)', '서울', '강남구', '수서동', '713번지', '수서현대벤처빌', '1123호']
['(62395)', '광주', '광산구', '장수동', '427번지']
['(14297)', '경기', '광명시', '옥길동', '133-4번지', '17']
['(21693)', '인천', '남동구', '고잔동', '731-10번지']
['(12638)', '경기', '여주시', '현암동', '586-22번지', '7호']
['(38216)', '경북', '경주시', '양남면', '환서리', '1011번지']
['(33504)', '충남', '보령시', '웅천읍', '수부리', '산6-2번지']
['(23011)', '인천', '강화군', '양사면', '인화리', '844-6번지']
['(26496)', '강원', '원주시', '귀래면', '귀래리', '1746번지']
['(25948)', '강원', '삼척시', 

In [ ]:
# df_sample에서 시도 태그추출('지번'컬럼이 들어감)
def address_matching_sample(address):
    li_ad = address.split(' ')
    try:
      if li_ad[1] in region_zip1.keys():  # 문자열 스페이스단위로 분리 후 위치 1에 '시도'가 있음.
          return region_zip1[li_ad[1]] # 딕셔너리의 value값 반환
      else:
          return region_zip1['전국']  # 해당사항 없는 경우 전국으로 추출 # 미기입이거나 외국이름이 적혀있는 경우 있음.
    except:
          return region_zip1['전국']

In [ ]:
# df_sample에서 시군구 태그 추출하는 함수
def address_matching_sample_low(address):
    li_ad = address.split(' ')
    try:
      if li_ad[2] in region_zip2.keys() :
          return region_zip2[li_ad[2]]
      else:
          return ''  # 이 부분 안들어가주면 에러날 수 있음.
    except:
      return ''

In [ ]:
def region_sample(region):  # 이곳에는 df_sample의 '지번' 값이 들어오게된다. 

    li = []  # 인덱스를 담을 list

    address1 = address_matching_sample(region)  # '시도' 태그추출함수
    address2 = address_matching_sample_low(region)  # '시군구' 태그추출함수

    for i in range(len(policy_df)):  #  policy_df갯수만큼  for문으로 돌림.
        if policy_df['추출_시군구1'][i] == ['']:  # 시군구 값이 존재하지 않는 경우
            if address1 in policy_df['추출_시도1'][i] :  # 시도 값만 맞춰서 반환
                li.append(policy_df.index[i])
        
        else:  # 시군구 값이 존재하는 경우
            if (address1 in policy_df['추출_시도1'][i]) & (address2 in policy_df['추출_시군구1'][i]):# 둘다 포함되었을 때만 인덱스값 추가
                li.append(policy_df.index[i]) 
    
    # df = policy_df.iloc[li,:] # li의 인덱스만 추출, 열은 전체 추출
    return li

In [ ]:
len(region_sample(df_sample['지번'][0])), len(region_sample(df_sample['지번'][400]))  # 잘 추출되고 있음.

(6, 5)

In [ ]:
region_sample(df_sample['지번'][400])

[4442, 4443, 4446, 4500, 4526]

In [ ]:
# 확인
df_sample['지번'][400]  # 서울 영등포구에 해당하는 정책 갯수는?

'(06634) 서울 서초구 서초1동 1657-5번지'

In [ ]:
lis =[]
for i in range(len(policy_df)):
    if policy_df['추출_시군구1'][i] == ['']:
        if (region_zip1['서울'] in policy_df['추출_시도1'][i]):
            lis.append(policy_df.index[i])
    else:
        if (region_zip1['서울'] in policy_df['추출_시도1'][i])&(region_zip2['서초구'] in policy_df['추출_시군구1'][i]):
            lis.append(policy_df.index[i])

print(policy_df.iloc[lis,:].shape) # lis의 인덱스만 추출, 열은 전체 추출
# 5개로 동일하게 추출

(5, 28)


### 종업원수 매칭

In [ ]:
df_sample['종업원수'].unique()

array(['1,589명', '17명', '67명', '65명', '1명', '62명', '74명', '명', '759명',
       '132명', '8명', '10명', '3명', '22명', '19명', '51명', '38명', '666명',
       '34명', '220명', '6명', '0명', '13명', '14명', '50명', '39명', '54명', '5명',
       '21명', '2명', '46명', '35명', '4명', '23명', '32명', '20명', '15명', '61명',
       '18명', '9명', '43명', '52명', '37명', '28명', '40명', '70명', '26명',
       '30명', '104명', '7명', '24명', '78명', '175명', '25명', '289명', '11명',
       '310명', '45명', '478명', '31명', '49명', '63명', '90명', '16명', '42명',
       '1,448명', '36명', '141명', '129명', '207명', '27명', '144명', '116명',
       '88명', '2,260명', '158명', '142명', '69명', '86명', '122명', '253명',
       '41명', '454명', '29명', '446명', '107명', '106명', '857명', '1,808명',
       '60명', '77명', '80명', '75명', '53명', '1,080명', '85명', '584명', '258명',
       '47명', '146명', '554명', '270명', '800명', '198명', '888명', '1,054명',
       '261명', '2,107명', '192명', '57명', '154명', '641명', '143명', '575명',
       '12명', '277명', '717명', '160명', '209명', '109명', '44명', '552

In [ ]:
df_sample[df_sample['종업원수']=='명'].shape   # 결측치 117row

(117, 28)

In [ ]:
# df_sample의 종업원수에 대해 int를 반환하는 함수 : 결측치 117row 에 대해서는 종업원수 관계없으므로 1 반환
import re
def em_num(x):
    numbers = re.sub(r'[^0-9]', '', x)
    if numbers != '' :
        return int(numbers)
    else:
        return 1

In [ ]:
# df_sample['종업원수'][i](아직 str값인 상태) 값을 받아서 해당되는 데이터프레임을 반환하는 함수
def employee_sample(df_num):
    sample_num = em_num(df_num)  # 종업원수 str형을 int로 변환
    sample_df = policy_df[(policy_df['추출_종업원수_최소']<= sample_num) & (sample_num <= policy_df['추출_종업원수_최대'])]
    return sample_df.index   # sample_df의 index리스트를 반환

In [ ]:
employee_sample(df_sample['종업원수'][10]).shape, employee_sample(df_sample['종업원수'][12]).shape

((9263,), (9523,))

In [ ]:
type(employee_sample(df_sample['종업원수'][10]))

pandas.core.indexes.numeric.Int64Index

### 설립기간 매칭


In [ ]:
# 설립기간 index를 반환하는 함수

from datetime import date

def foundate_sample(df_num):
    if df_num == '-': # 값이 없을 때는 모든 인덱스 반환
        return list(policy_df.index)
    else:
        fd_yrs = date.today().year - df_num.year # 이번 년도에서 설립일자를 뺀 기간 
        sample_df = policy_df[(policy_df['추출_설립기간_최소'].astype(int)<= fd_yrs) & \
                              (fd_yrs <= policy_df['추출_설립기간_최대'].astype(int))]
        return sample_df.index

### 주요 업종 매칭

In [ ]:
def product_sample(df_num):
    index = policy_df.index
    if df_num == '-': # 값이 없을 때는 모든 인덱스 반환
        return list(policy_df.index)
    else:
        field = df_num[2:4]

        policy_df['주요업종_filtered'] = ['1' if (str(field) in x) \
                                      else '0' for x in policy_df['추출_주요업종']]
        condition = policy_df['주요업종_filtered']=='1'
        product_indices = index[condition]

        return product_indices
    

### *적합한 df 추출하기

In [ ]:
df_sample.shape

(509, 28)

In [ ]:
# 인덱스 리스트간 교집합구하기
def sample_matching(x): # x = df_sample의 인덱스

    emp_lst = employee_sample(df_sample['종업원수'][x])  
    region_lst = region_sample(df_sample['지번'][x])
    foundate_lst = foundate_sample(df_sample['설립일자'][x])
    product_lst = product_sample(df_sample['업종(10차)'][x])

    # 인덱스리스트 교집합 구하기
    intersection = list(set(emp_lst) & set(region_lst) & set(foundate_lst) & set(product_lst)) 

    # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df  #     set1 = [x for x in emp_lst if x in region_lst]


In [ ]:
# 인덱스 리스트간 교집합구하기
def sample_matching2(x): # x = df_sample의 인덱스

    emp_lst = employee_sample(df_sample['종업원수'][x])  
    region_lst = region_sample(df_sample['지번'][x])
    foundate_lst = foundate_sample(df_sample['설립일자'][x])
    product_lst = product_sample(df_sample['업종(10차)'][x])

#     # 인덱스리스트 교집합 구하기
    intersection = list(set(emp_lst) & set(region_lst)& set(foundate_lst) & set(product_lst))

#     # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df['정책ID'].values.tolist()

In [ ]:
a = [1,2,3,4,5]
b = [2,3,6,7,5]
a or b

[1, 2, 3, 4, 5]

In [ ]:
df_sample['추출_정책아이디'] = df_sample.reset_index()['index'].apply(sample_matching2)

In [ ]:
df_sample['추출_정책아이디']

In [ ]:
df_sample.columns

In [ ]:
sample_matching(8).shape, sample_matching(6).shape, sample_matching(0).shape

In [ ]:
df_sample.loc[[0]][['기업체명', '도로명', '설립형태', '업종(10차)']]

In [ ]:
sample_matching(0)[['정책ID', '서비스명', '지원대상']]

In [ ]:
df_sample.loc[[8]][['기업체명', '도로명', '설립형태', '업종(10차)']]

In [ ]:
sample_matching(8)[['정책ID', '서비스명', '지원대상']]

In [ ]:
len(temp_list)

In [ ]:
sample_matching(8)[['정책ID']]

In [ ]:
sample_matching2(8)

In [ ]:
df_sample.columns

In [6]:
# 엑셀로 저장하기
# df_sample.to_excel("/content/drive/MyDrive/1팀/sample기업데이터 매칭완료.xlsx", sheet_name='new_name', index=False)

# 엑셀파일 읽기
import pandas as pd
df_sample = pd.read_excel("/content/drive/MyDrive/1팀/기업 데이터 (형태+학습) 정책 추천 완료.xlsx")

In [9]:
df_sample.columns

Index(['Unnamed: 0', '섹터', '기업체명', '영문기업명', '사업자번호', '법인(주민)번호', '대표자명',
       '종업원수', '설립형태', '설립일자', '기업형태', '기업규모', '전화번호', '팩스번호', '홈페이지', '이메일',
       '결산월', '기업공개일자', '도로명', '지번', '업종(10차)', '업종(9차)', '주요제품(상품)',
       '무역업허가번호', '소속그룹', '주채권기관', '당좌거래은행', '휴폐업정보', '법인등기정보', '추천 정책'],
      dtype='object')

In [12]:
for i in df_sample['추천 정책']:
    if len(i) >10:
        print(len(i))

12
55
113
36
14
14
14
36
14
66
19
27
12
28
14
21
424
42
13
14
21
21
14
20
42
196
35
94
28
35
28
378
28
14
12
55
113
36
14
14


## df_final 매칭시스템

In [ ]:
df_final.fillna('', inplace=True)

### 지역매칭

In [ ]:
# # 정책데이터 결측치 제거하고 리스트로 만들기
# policy_df['추출_시군구'] = policy_df['추출_시군구'].fillna('')

# policy_df['추출_시도1'] = policy_df['추출_시도'].apply(lambda x : x.split(', '))
# policy_df['추출_시군구1'] = policy_df['추출_시군구'].apply(lambda x:x.split(', '))

In [ ]:
df_final.head(1)

In [ ]:
df_final['기업기본주소'].isnull().sum()

In [ ]:
df_final['기업상세주소'].isnull().sum()

In [ ]:
# df_final의 시도태그 추출
def address_match_final(address):
    li_ad = address.split(' ')
    if li_ad[0] in region_zip1.keys() :
        return region_zip1[li_ad[0]]
    else:
        return region_zip1['전국']

In [ ]:
# df_final의 시군구 태그 추출
def address_match_final_low(address):
    li_ad = address.split(' ')
    if li_ad[1] in region_zip2.keys() :
        return region_zip2[li_ad[1]]
    else:
        return ''   # ★ 이부분 필수

In [ ]:
def region_final(region):  # 이곳에는 df_sample의 '지번' 값이 들어오게된다. 

    li = []  # 인덱스를 담을 list

    address1 = address_match_final(region)  # '시도' 태그추출함수
    address2 = address_match_final_low(region)  # '시군구' 태그추출함수

    for i in range(len(policy_df)):  #  policy_df갯수만큼  for문으로 돌림.
        if policy_df['추출_시군구1'][i] == ['']:  # policy_df에 시군구 값이 존재하지 않는 경우
            if address1 in policy_df['추출_시도1'][i] :  # 시도 값만 맞춰서 반환
                li.append(policy_df.index[i])
        
        else:  # policy_df에 시군구 값이 존재하는 경우
            if (address1 in policy_df['추출_시도1'][i]) & (address2 in policy_df['추출_시군구1'][i]):# 둘다 포함되었을 때만 인덱스값 추가 # ★여기서 address2가 None이 들어올 때 error
                li.append(policy_df.index[i]) 
    
    # df = policy_df.iloc[li,:] # li의 인덱스만 추출, 열은 전체 추출
    return li

In [ ]:
len(region_final(df_final['기업기본주소'][0])), len(region_final(df_final['기업기본주소'][99]))  # 잘 추출되고 있음.

### 종업원수 매칭


In [ ]:
df_final['종업원수'].unique()  # type = int 라서 따로 변환필요x

In [ ]:
df_final[df_final['종업원수']==1673]

In [ ]:
df_final['종업원수'].isnull().sum()  # 결측치 없음

In [ ]:
# df_sample['종업원수'][i](int) 값을 받아서 해당되는 데이터프레임을 반환하는 함수
def employee_final(df_num):
    if df_num == 0:  # 만약 0명이라면 직원수 무관 정책에 해당하니까 1명부터에 해당하는 기업(1인기업 등)도 추출되도록 1로 변환
        df_num = 1
    sample_df = policy_df[(policy_df['추출_종업원수_최소']<= df_num) & (df_num <= policy_df['추출_종업원수_최대'])]
    return sample_df.index

In [ ]:
employee_final(df_final['종업원수'][0]).shape, employee_final(df_final['종업원수'][9086]).shape

### 설립기간 매칭


In [ ]:
df_final['기업설립일자'][11051]

In [ ]:
# 설립기간 index를 반환하는 함수

from datetime import date

def foundate_final(df_num):
    if str(df_num) == '': # 값이 없을 때는 모든 인덱스 반환   # fillna 적용
        return list(policy_df.index)
    else:
        fd_yrs = date.today().year - int(df_num//10000) # 이번 년도에서 설립일자를 뺀 기간 
        sample_df = policy_df[(policy_df['추출_설립기간_최소'].astype(int)<= fd_yrs) & (fd_yrs <= policy_df['추출_설립기간_최대'].astype(int))]
        return sample_df.index
    

### 주요 업종 매칭

In [ ]:
def product_final(df_num):
    index = policy_df.index
    field = ''
    if df_num == '':   # ★결측값 따로설정안해주면 에러
        return list(policy_df.index)
    elif str(df_num).isdigit() == False:  # ★업종 dtypes = object. 문자열 셀은 따로 처리
        return list(policy_df.index)
    else:
        if str(int(df_num)).isdigit():
            if int(df_num) < 10:
                field = '0' + str(df_num)
            else:
                field = str(df_num)[:2]
        else:
            return list(policy_df.index)

    policy_df['주요업종_filtered'] = ['1' if (str(field) in x) else '0' for x in policy_df['추출_주요업종']]
    condition = policy_df['주요업종_filtered']=='1'
    product_indices = index[condition]

    return product_indices
    

In [ ]:
product_lst = product_final(df_final['업종'][7])

In [ ]:
len(product_lst)

In [ ]:
df_final.dtypes

### *적합한 df 추출

In [ ]:
df_final.shape

In [ ]:
# 인덱스 리스트간 교집합구하기
def final_matching(x): # x = df_final의 인덱스
    # policy_df의 적합한 인덱스 리스트를 추출
    emp_lst = employee_final(df_final['종업원수'][x])  
    region_lst = region_final(df_final['기업기본주소'][x])
    foundate_lst = foundate_final(df_final['기업설립일자'][x])
    product_lst = product_final(df_final['업종'][x])

    # 인덱스리스트 교집합 구하기
    intersection = list(set(emp_lst) & set(region_lst) & set(foundate_lst) & set(product_lst))

    # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df

In [ ]:
# 인덱스 리스트간 교집합구하기
def final_matching2(x): # x = df_sample의 인덱스

    emp_lst = employee_final(df_final['종업원수'][x]) 
    region_lst = region_final(df_final['기업기본주소'][x])
    foundate_lst = foundate_final(df_final['기업설립일자'][x])
    product_lst = product_final(df_final['업종'][x])

#     # 인덱스리스트 교집합 구하기
    intersection = list(set(emp_lst) & set(region_lst) &set(foundate_lst) & set(product_lst))

#     # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df['정책ID'].values.tolist()

In [ ]:
final_matching(10).shape, final_matching(0).shape, final_matching(9999).shape

In [ ]:
df_final[['종업원수', '기업기본주소', '기업설립일자', '업종']].isnull().sum()

In [ ]:
df_final[57]

In [ ]:
df_final['추출_정책아이디'] = df_final.reset_index()['index'].apply(final_matching2)

In [ ]:
df_final.columns

In [ ]:
df_final['추출_정책아이디']

In [8]:
# 엑셀로 저장하기
# df_final.to_excel("/content/drive/MyDrive/1팀/final기업데이터 매칭완료.xlsx", sheet_name='new_name', index=False)

# 엑셀파일 읽기
import pandas as pd
df_final = pd.read_excel("/content/drive/MyDrive/1팀/final기업데이터 매칭완료.xlsx")

In [14]:
df_final.columns

Index(['회계감사인명', '감사보고서의견내용', '기준일자', '사업자등록번호', '법인구분코드', '법인구분코드명', '법인영문명',
       '기업체명', '법인등록시장구분코드', '법인등록시장구분코드명', '법인등록번호', '종업원평균근속기간내용', '기업기본주소',
       '기업상세주소', '종업원수', '기업설립일자', '기업팩스번호', '홈페이지', '기업코스닥상장폐지일자',
       '기업코스닥상장일자', '기업KONEX상장폐지일자', '기업KONEX상장일자', '기업주요사업명', '기업주거래은행명',
       '기업구우편번호', '기업공시회사명', '기업1인평균급여금액', '기업대표자성명', '기업결산월', '기업전화번호',
       '기업거래소상장폐지일자', '기업거래소상장일자', '금융감독원법인변경일시', '금융감독원법인고유번호', '업종',
       '중소기업여부', '추출_정책아이디'],
      dtype='object')

In [27]:
li = []
for i in range(len(df_final['추출_정책아이디'])):
    if len(df_final['추출_정책아이디'][i]) > 100:
        # print('추출된 정책아이디 개수: ', len(df_final['추출_정책아이디'][i]))
        li.append(df_final.index[i])
        # print(df_final.loc[[i],['종업원수', '기업기본주소', '기업설립일자', '업종']])
        print(df_final.loc[[i],['업종']])
# 매칭정책갯수가 많은 기업의 공통점은 특별히 발견되지 않음.

      업종
132  NaN
        업종
155  22211
      업종
200  NaN
        업종
219  20119
      업종
226  NaN
        업종
322  64911
       업종
402  1012
        업종
471  15110
      업종
472  NaN
        업종
502  15190
    업종
511  F
      업종
544  NaN
        업종
586  20411
        업종
603  64999
      업종
624  NaN
       업종
629  4111
        업종
725  41210
        업종
731  15110
     업종
732  42
        업종
847  20129
        업종
977  28303
         업종
1090  64999
         업종
1135  15110
         업종
1281  20119
         업종
1313  20121
         업종
1348  29199
       업종
1355  NaN
       업종
1440  NaN
         업종
1470  29172
         업종
1508  15220
        업종
1534  2811
         업종
1535  41112
       업종
1641  NaN
       업종
1702  NaN
         업종
1750  23232
       업종
1834  NaN
       업종
1889  NaN
       업종
1949  NaN
        업종
1991  7512
         업종
1992  41210
       업종
2027  NaN
       업종
2082  NaN
        업종
2126  4112
         업종
2145  41112
       업종
2176  NaN
         업종
2203  64201
         업종
2205  64201
  

In [22]:
df_final.iloc[li,:][['종업원수', '기업기본주소', '기업설립일자', '업종']]

,종업원수,기업기본주소,기업설립일자,업종
132,0,광주광역시 북구 양산동 405-13,NaN,NaN
155,0,충청북도 청원군 남이면 구미리 41번지,19931222.0,22211
200,0,경상북도 김천시 응명동 1016,NaN,NaN
219,0,경기도 양주군 은현면 용암리 73,19900109.0,20119
226,0,경기도 화성시 병점동 489-1,NaN,NaN
...,...,...,...,...
10284,0,서울특별시 영등포동2가 28-130,19600205.0,7121
10351,0,경상남도 마산시 회원구 봉암동 666-158번지,19560127.0,13212
10609,0,충청남도 당진군 당진읍 구룡리산44,20040401.0,7122
10901,0,인천광역시 남동구 고잔동 707-10,NaN,NaN


In [23]:
df_final[df_final['종업원수']==0].shape

(11035, 37)

In [24]:
df_final.shape

(11056, 37)

## df_dart 매칭시스템

### 지역매칭

In [ ]:
df_final.columns

Index(['회계감사인명', '감사보고서의견내용', '기준일자', '사업자등록번호', '법인구분코드', '법인구분코드명', '법인영문명',
       '기업체명', '법인등록시장구분코드', '법인등록시장구분코드명', '법인등록번호', '종업원평균근속기간내용', '기업기본주소',
       '기업상세주소', '종업원수', '기업설립일자', '기업팩스번호', '홈페이지', '기업코스닥상장폐지일자',
       '기업코스닥상장일자', '기업KONEX상장폐지일자', '기업KONEX상장일자', '기업주요사업명', '기업주거래은행명',
       '기업구우편번호', '기업공시회사명', '기업1인평균급여금액', '기업대표자성명', '기업결산월', '기업전화번호',
       '기업거래소상장폐지일자', '기업거래소상장일자', '금융감독원법인변경일시', '금융감독원법인고유번호', '업종',
       '중소기업여부'],
      dtype='object')

In [ ]:
df_dart.head()

,결산월(MM),주소,사업자등록번호,대표자명,"법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)",corp_code,정식명칭,영문명칭,설립일(YYYYMMDD),팩스번호,홈페이지,업종번호,IR홈페이지,법인등록번호,message,전화번호,status,주식 종목코드,종목명 또는 약식명칭
0,12.0,충청남도 천안시 청당동 419-12,3128134722,김상규,E,434003,(주)다코,Daco corporation,19970611.0,041-563-6808,없음,25931,NaN,1615110021778,정상,041-565-1800,0,NaN,다코
1,12.0,강원도 원주시 원동 303,2248105027,이광선,E,434456,일산약품(주),"ILSAN PHARMACEUTICAL CO.,LTD.",19910615.0,033-763-6433,NaN,21101,NaN,1412110004641,정상,033-763-7535,0,NaN,일산약품
2,11.0,서울특별시 중구 태평로2가 120,1048177488,정진형,E,430964,굿앤엘에스주식회사,"Good & LS Co.,Ltd.",20021213.0,NaN,NaN,64999,NaN,1101112672875,정상,02-6263-8185,0,NaN,굿앤엘에스
3,12.0,제주특별자치도 북제주군 한림읍 상명리 587,6168120149,정우석,E,432403,한라판지(주),"HANLA PARKING CO.,LTD",19970601.0,064-796-8003,NaN,17123,NaN,2201110019993,정상,064-796-8001,0,NaN,한라판지
4,10.0,서울특별시 영등포구 여의도동 36-3,107-86-08501,배경오,E,388953,크레디피아제이십오차유동화전문회사,Credipia 25th Asset Securitization Specialty L.L.C,20011121.0,784-9869,NaN,64999,NaN,1101140036118,정상,07-768-7911,0,NaN,크레디피아제이십오차유동화전문회사


In [ ]:
df_dart[df_dart['주소'].isnull()]

,결산월(MM),주소,사업자등록번호,대표자명,"법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)",corp_code,정식명칭,영문명칭,설립일(YYYYMMDD),팩스번호,홈페이지,업종번호,IR홈페이지,법인등록번호,message,전화번호,status,주식 종목코드,종목명 또는 약식명칭
137,12.0,NaN,NaN,남우길,E,290816,대우종합기계주식회사(가칭),NaN,NaN,NaN,NaN,NaN,NaN,NaN,정상,NaN,0,NaN,대우종합기계(가칭)
138,12.0,NaN,NaN,남우길,E,291532,대우조선공업주식회사(가칭),NaN,NaN,NaN,NaN,NaN,NaN,NaN,정상,NaN,0,NaN,대우조선공업(가칭)
19297,12.0,NaN,17999,"Tak Wai, Wong",E,544027,"LCF INVESTMENTS, LTD","LCF INVESTMENTS, LTD",20050228.0,-,-,64209,NaN,NaN,정상,-,0,NaN,"LCFINVESTMENTS,LTD"


In [ ]:
# 시도 매칭
df_dart['주소'].fillna('', inplace=True)
df_dart['주소'].fillna('', inplace=True)

In [ ]:
# df_final의 시도태그 추출
def address_match_final(address):
    li_ad = address.split(' ')
    try:
      if li_ad[0] in region_zip1.keys() :
          return region_zip1[li_ad[0]]
      else:
          return region_zip1['전국']
    except:
      return region_zip1['전국']

In [ ]:
# df_final의 시군구 태그 추출
def address_match_final_low(address):
    li_ad = address.split(' ')
    try:
      if li_ad[1] in region_zip2.keys() :
          return region_zip2[li_ad[1]]
      else:
          return ''   # ★ 이부분 필수
    except:
      return ''

In [ ]:
def region_dart(region):  # 이곳에는 df_sample의 '지번' 값이 들어오게된다. 

    li = []  # 인덱스를 담을 list

    address1 = address_match_final(region)  # '시도' 태그추출함수
    address2 = address_match_final_low(region)  # '시군구' 태그추출함수

    for i in range(len(policy_df)):  #  policy_df갯수만큼  for문으로 돌림.
        if policy_df['추출_시군구1'][i] == ['']:  # 시군구 값이 존재하지 않는 경우
            if address1 in policy_df['추출_시도1'][i] :  # 시도 값만 맞춰서 반환
                li.append(policy_df.index[i])
        
        else:  # 시군구 값이 존재하는 경우
            if (address1 in policy_df['추출_시도1'][i]) & (address2 in policy_df['추출_시군구1'][i]):# 둘다 포함되었을 때만 인덱스값 추가
                li.append(policy_df.index[i]) 
    
    # df = policy_df.iloc[li,:] # li의 인덱스만 추출, 열은 전체 추출
    return li

In [ ]:
len(region_dart(df_dart['주소'][0])), len(region_dart(df_dart['주소'][99]))  # 잘 추출되고 있음.

(25, 8)

### 설립기간 매칭


In [ ]:
# 설립기간 index를 반환하는 함수

from datetime import date

def foundate_final(df_num):
    if str(df_num) == 'nan': # 값이 없을 때는 모든 인덱스 반환
        return list(policy_df.index)
    else:
        fd_yrs = date.today().year - int(df_num//10000) # 이번 년도에서 설립일자를 뺀 기간 
        sample_df = policy_df[(policy_df['추출_설립기간_최소'].astype(int)<= fd_yrs) & (fd_yrs <= policy_df['추출_설립기간_최대'].astype(int))]
        return sample_df.index
    

### 주요 업종 매칭

In [ ]:
def product_final(df_num):
    index = policy_df.index
    field = ''
    if df_num == '':   # ★결측값 따로설정안해주면 에러
        return list(policy_df.index)
    elif str(df_num).isdigit() == False:  # ★업종 dtypes = object. 문자열은 처리
        return list(policy_df.index)
    else:
        if str(int(df_num)).isdigit():
            if int(df_num) < 10:
                field = '0' + str(df_num)
            else:
                field = str(df_num)[:2]
        else:
            return list(policy_df.index)

    policy_df['주요업종_filtered'] = ['1' if (str(field) in x) else '0' for x in policy_df['추출_주요업종']]
    condition = policy_df['주요업종_filtered']=='1'
    product_indices = index[condition]

    return product_indices

### 종업원수 없음(df_dart)

### *적합한 df 추출

In [ ]:
# 인덱스 리스트간 교집합구하기
def dart_matching(x): # x = df_dart의 인덱스
    # policy_df의 적합한 인덱스 리스트를 추출
    region_lst = region_dart(df_dart['주소'][x])
    foundate_lst = foundate_final(df_final['기업설립일자'][x])
    product_lst = product_final(df_final['업종'][x])

    # 인덱스리스트 교집합 구하기
    intersection = list((set(region_lst) & set(foundate_lst)) & set(product_lst))

    # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df

In [ ]:
dart_matching(10).shape, dart_matching(0).shape, dart_matching(508).shape

((0, 28), (19, 28), (0, 28))

In [ ]:
# 인덱스 리스트간 교집합구하기
def dart_matching2(x): # x = df_sample의 인덱스

    region_lst = region_dart(df_dart['주소'][x])
    foundate_lst = foundate_final(df_dart['설립일(YYYYMMDD)'][x])
    product_lst = product_final(df_dart['업종번호'][x])

#     # 인덱스리스트 교집합 구하기
    intersection = list(set(region_lst) & set(foundate_lst) & set(product_lst))

#     # 적합한 row 추출
    df = policy_df.iloc[intersection,:]   # policy_df에서 교집합 인덱스에 해당되는 것 반환

    return df['정책ID'].values.tolist()

In [ ]:
df_dart['추출_정책아이디'] = df_dart.reset_index()['index'].apply(dart_matching2)

In [ ]:
df_dart.columns

In [ ]:
# 엑셀로 저장하기
df_dart.to_excel("/content/drive/MyDrive/wello/Wello AIB 1팀 제출자료 (Jan 19, 2022)/기업 데이터 (dart) 정책 추천 완료.xlsx", index=False)

df_dart.head()